<a href="https://colab.research.google.com/github/vifirsanova/OCR_Glossary/blob/main/workbook_collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### ноутбук будет дополнен

# Импорт библиотек

In [2]:
# использование цвета для вывода на экран
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
#from pathlib import Path
#import re
#from colorama import Fore, Style
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import word_tokenize 
from gensim.utils import simple_preprocess

# Загрузка нескольких файлов

* загружаем файлы из GitHub
* соединяем их в одной переменной

In [4]:
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Geology.txt
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Physical_Geology_2nd_Ed.txt
!wget https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Lumen-Learning-Geology.txt

--2023-02-01 12:23:32--  https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Geology.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207953 (203K) [text/plain]
Saving to: ‘Geology.txt’

Geology.txt         100%[===================>] 203.08K  --.-KB/s    in 0.03s   

2023-02-01 12:23:32 (7.87 MB/s) - ‘Geology.txt’ saved [207953/207953]

--2023-02-01 12:23:32--  https://raw.githubusercontent.com/vifirsanova/OCR_Glossary/main/Physical_Geology_2nd_Ed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1268676 (1.2M) [t

In [5]:
# проверяем наличие загруженных файлов в директории
# с помощью консольной команды ls

!ls

Geology.txt		    Physical_Geology_2nd_Ed.txt
Lumen-Learning-Geology.txt  sample_data


In [6]:
# прочитываем все файлы с помощью метода read_text из библиотеки Path
# мы можем выполнить это параллельно, 
# то есть за одну команду одновременно обработать несколько файлов

text0, text1, text2 = Path('Geology.txt').read_text(), Path('Lumen-Learning-Geology.txt').read_text(), Path('Physical_Geology_2nd_Ed.txt').read_text()

# посмотрим на часть одного из текстов

text2[:900]

'https://opentextbc.ca/physicalgeology2ed/\nPhysical Geology - 2nd Edition\nPhysical Geology - 2nd Edition\nSteven Earle\nKarla Panchuk\nBCcampus\nVictoria, B.C.\nIcon for the Creative Commons Attribution 4.0 International License\n\nPhysical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.\n\n© 2019 Steven Earle\n\nThe CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:\n\nPhysical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.\nIf you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:\n\nDownload for free from the B.C. Open Collection'

In [7]:
# сохраним этот отрывок в отдельную переменную для будущих экспериментов c RegEx

test = text2[:900]

Какие артефакты бросаются нам в глаза в этом тексте?

In [8]:
# объединим все файлы в один и посмотрим его статистику

text = " ".join([text0, text1, text2])

print(f'Статистическое описание сырого документа\n\n{Fore.RED}Длина документа:{Style.RESET_ALL} {len(text)} символов\n\
{Fore.RED}Приблизительное количество слов в документе:{Style.RESET_ALL} {len(text.split(" "))}')

Статистическое описание сырого документа

Длина документа: 2452019 символов
Приблизительное количество слов в документе: 375467


Какие статистические данные по сырому тексту нам могут быть интересны?

Как их получить?

# Предобработка данных

## Чистка данных с помощью RegEx

Вспомним отрывок одного из текстов:

```
https://opentextbc.ca/physicalgeology2ed/\nPhysical Geology - 2nd Edition\nPhysical Geology - 2nd Edition\nSteven Earle\nKarla Panchuk\nBCcampus\nVictoria, B.C.\nIcon for the Creative Commons Attribution 4.0 International License\n\nPhysical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.\n\n© 2019 Steven Earle\n\nThe CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:\n\nPhysical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.\nIf you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:\n\nDownload for free from the B.C. Open Collection

```

Мы обратили внимание на некоторые артефакты:

* ссылки
* абзацные отступы
* специальные символы, например `©`
* ???

Проверим, верна ли наша гипотеза.




In [9]:
# ссылки

link_pattern = re.compile(r'https?://\S+|www\.\S+')
links = re.findall(link_pattern, text)

print(f'{Fore.RED}Количество обнаруженных ссылок:{Style.RESET_ALL} {len(links)}\n{Fore.RED}Образцы ссылок:{Style.RESET_ALL} {links[:3]}')

# специальные символы

symbol_pattern = re.compile(r'[^a-zA-Z0-9 .,:;…()!?\-—]+')
symbols = re.findall(symbol_pattern, text)

print(f'\n{Fore.RED}Количество обнаруженных специальных символов:{Style.RESET_ALL} {len(symbols)}\n{Fore.RED}Образцы уникальных символов:{Style.RESET_ALL} {set(symbols[500:1000])}')

# любые символы

any_symbol_pattern = re.compile(r'[^a-zA-Z ]+')
all_symbols = re.findall(any_symbol_pattern, text)

print(f'\n{Fore.RED}Количество обнаруженных символов:{Style.RESET_ALL} {len(all_symbols)}\n{Fore.RED}Образцы уникальных символов:{Style.RESET_ALL} {set(all_symbols[500:600])}')

Количество обнаруженных ссылок: 341
Образцы ссылок: ['https://opengeology.org/textbook/1-understanding-science/', 'https://www.columbiabasin.edu/.', 'http://www.ck12.org/book/CK-12-Earth-Science-For-Middle-School/section/1.2/.']

Количество обнаруженных специальных символов: 23382
Образцы уникальных символов: {'+', '–', '\n\n\n\n\n\n', '\n\n\n\n\n', '°/', '·', '/', '“', '’', '[', '°', 'ö', '>', ']', 'ᐧ', '”', '%', '\n', '&', '\n\n\n', 'ʻ', 'í', "'", '‘', '\n\n\n\n', '•', 'ʻā', '<'}

Количество обнаруженных символов: 76614
Образцы уникальных символов: {'2.\n', '-', ',', '.\n', '(', '.', '—', ';', '.\n1.5.3', '.\n1.5.2', ').\n', '),', '\n', '’', ':'}


Мы хотим проверить, какое влияние окажет удаление знаков препинания на результат работы нашей программы.

Для этого создадим три функции для чистки текста: 
- первая сохраняет знаки препинания и цифры
- вторая очищает текст от знаков препинания
- третья удаляет все символы кроме букв

In [10]:
def clean(text):
  ### сохраняет знаки препинания и цифры ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # оставляем знаки препинания и цифры
  text = re.sub('[^a-zA-Z0-9 .,:;…()!?\-—]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

def full_clean(text):
  ### очищает текст от знаков препинания ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # оставляем цифры
  text = re.sub('[^a-zA-Z0-9 ]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

def digits_clean(text):
  ### удаляет все символы кроме букв ###

  # удаляем из текста ссылки 
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  
  # удаляем специальные символы
  # удаляем цифры
  text = re.sub('[^a-zA-Z ]+', ' ', text)

  # возвращаем текст, очищенный от лишних пробелов
  # которые появились в результате обработки
  return re.sub('\s+', ' ', text)

In [11]:
cleaned, full_cleaned, digits_cleaned = clean(test), full_clean(test), digits_clean(test)

print(f'Пример очистки текста\n\n{Fore.RED}Тестовый образец:{Style.RESET_ALL}\n"{test}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции clean (сохраняет знаки препинания и цифры){Style.RESET_ALL}\n"{cleaned}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции full_clean (очищает текст от знаков препинания){Style.RESET_ALL}\n"{full_cleaned}"\n\n \
{Fore.RED}* Образец, очищенный с помощью функции digits_clean (удаляет все символы кроме букв){Style.RESET_ALL}\n"{digits_cleaned}"')

Пример очистки текста

Тестовый образец:
"https://opentextbc.ca/physicalgeology2ed/
Physical Geology - 2nd Edition
Physical Geology - 2nd Edition
Steven Earle
Karla Panchuk
BCcampus
Victoria, B.C.
Icon for the Creative Commons Attribution 4.0 International License

Physical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.

© 2019 Steven Earle

The CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows:

Physical Geology – 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence.
If you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost:

Download for free from the B.C. Open Collection"

 * Образец, очищенный с помощью функции clean (сохраняе

## Токенизация

Рассмотрим несколько способов токенизации и создадим для каждого отдельную функцию.

In [24]:
# простая токенизация

def simple_tokenizer(text):
  # принимает на вход текст
  # приводит текст к нижнем регистру с помощью lower()
  # "рубит" текст на элементы, разделенные пробелом (слова)
  return text.lower().split(' ')

# токенизация методами NLTK

def nltk_tokenizer(text):
  # применяет метод word_tokenize из nltk к тексту
  return word_tokenize(text)

# токенизация с помощью gensim

def gensim_tokenizer(text, deacc):
  # применяет метод simple_preprocess из gensim к тексту
  # encode: кодировка текст
  # deacc удаляет знаки препинания, принимает значения True / False
  return simple_preprocess(str(text).encode('utf-8'), deacc)

In [29]:
tokenized, nltk_tokenized, gensim_tokenized = simple_tokenizer(cleaned), nltk_tokenizer(cleaned), gensim_tokenizer(cleaned,True)

print(f'Пример токенизации текста\n\n{Fore.RED}Тестовый образец:{Style.RESET_ALL}\n"{cleaned}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции simple_tokenizer {Style.RESET_ALL}\n"{tokenized}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции nltk_tokenizer {Style.RESET_ALL}\n"{nltk_tokenized}"\n\n \
{Fore.RED}* Образец, токенизированный с помощью функции gensim_tokenizer {Style.RESET_ALL}\n"{gensim_tokenized}"')

Пример токенизации текста

Тестовый образец:
" Physical Geology - 2nd Edition Physical Geology - 2nd Edition Steven Earle Karla Panchuk BCcampus Victoria, B.C. Icon for the Creative Commons Attribution 4.0 International License Physical Geology - 2nd Edition by Steven Earle is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted. 2019 Steven Earle The CC licence permits you to retain, reuse, copy, redistribute, and revise this book—in whole or in part—for free providing the author is attributed as follows: Physical Geology 2nd Edition by Steven Earle is used under a Creative Commons Attribution 4.0 International Licence. If you redistribute all or part of this book, it is recommended the following statement be added to the copyright page so readers can access the original book at no cost: Download for free from the B.C. Open Collection"

 * Образец, токенизированный с помощью функции simple_tokenizer 
"['', 'physical', 'geology', '-', '2

In [12]:
### TODO:
### новые стоп-слова
### би- и триграммные модели gensim
### посчитать TF 
### сделать частотный словарь 